In [89]:
import pulp

In [90]:

soc = [100,100,100]
n = len(soc)
stations = [20,0,40,0,100]
charge_points_list = [1,1,1,1]
#stations = [90,120,150]
charging_points = [[0,1],[2,3]]
speed = 10
charging_rate_station = 5
ex_charging_rate_bess = 10
arrival_time = [0,10,15]

T = 1000
bess_capacity = 300
bess_init = [250,250]
kms_to_soc = 1

for i in range(len(stations)):
    stations[i]=stations[i]*kms_to_soc
speed = speed**kms_to_soc
print(speed)
print(stations)

time_discre = 1
for i in range(len(arrival_time)):
    arrival_time[i] *= time_discre
speed = speed/time_discre
charging_rate_station = charging_rate_station/time_discre
ex_charging_rate_bess = ex_charging_rate_bess/time_discre



10
[20, 0, 40, 0, 100]


In [97]:
#initiate the model
cs = pulp.LpProblem("station",pulp.LpMinimize)

vehicles=[]
s_names = []
#add variables
soc_vehicle = {}
arrival_time_vehicle = {}
for i in range(n):
    vehicles.append(f"V{i}")
    soc_vehicle[f"V{i}"] = soc[i]
    arrival_time_vehicle[f"V{i}"] = arrival_time[i]
m = len(stations)-1
bess_init_name = {}
for i in range(m):
    s_names.append(f"station{i+1}")

grouped_stations = {}
k = 0
i=0 
while i in range(len(charging_points)):
    grouped_stations[s_names[k]] = []
    temp = s_names[k]
    for point in charging_points[i]:
        grouped_stations[temp].append(s_names[point])
        k+=1
    i+=1
for i,grouped_station in enumerate(list(grouped_stations.keys())):
    bess_init_name[grouped_station] = bess_init[i]
    
print(grouped_stations)
print(s_names)
print(bess_init_name)
M = 100000
print(vehicles)
#defining 
a = {}
s = {}
u = {}
d = {}
o = {}
sb = {}
b = {}
bc = {}
charge = {}
bac = {}

for station in s_names:
    #arrival time at stations
    a[station] = pulp.LpVariable.dict(f"Arrival Time {station}",[vehicle for vehicle in vehicles],lowBound=0,upBound=T,cat=pulp.LpInteger)
    
    # servicing time at stations
    s[station] = pulp.LpVariable.dict(f"Service Time {station}",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
    
    #initial charging time at sations
    u[station] = pulp.LpVariable.dict(f"initial charge time {station}",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
    
    #final charge time at stations
    d[station] = pulp.LpVariable.dict(f"final charge time {station}",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
    
    # sigma- temporal ordering of vehicles
    o[station] = pulp.LpVariable.dict(f"sigma {station}",[vehicle1+vehicle2 for vehicle1 in vehicles for vehicle2 in vehicles if vehicle1 != vehicle2],lowBound=0,cat=pulp.LpBinary)
    
    # to see if vehicle is gonna use the respective station
    sb[station] = pulp.LpVariable.dict(f"Service Time {station} binary",[vehicle for vehicle in vehicles],cat=pulp.LpBinary)
    
    #bess servicing times at stations
    b[station] = pulp.LpVariable.dict(f"BESS service Time {station}",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
    
    
    
    
for grouped_station in grouped_stations:
    print(grouped_station)
    #amount to charge
    bac[grouped_station] = pulp.LpVariable.dict(f"BESS charge for vehicle {grouped_station}",[vehicle for vehicle in vehicles],cat=pulp.LpInteger)
    # charge in bess available for vehicles
    bc[grouped_station] = pulp.LpVariable.dict(f"BESS charge {grouped_station}",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
    # if bess reached max charge or not
    charge[grouped_station] = pulp.LpVariable.dict(f"reached max charge {grouped_station}",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpBinary)
print(bac,bc,charge)
    
    
#finding arrival times for each station.
for i in range(len(s_names)):
    for vehicle in vehicles:
        if i == 0:
            cs += a[s_names[i]][vehicle] == arrival_time_vehicle[vehicle]+(stations[0]/speed)
        else:
            cs += a[s_names[i]][vehicle] == (stations[i]/speed) + d[s_names[i-1]][vehicle]

#ensuring the vehicle leaves by the end of the end of the max Time
for vehicle in vehicles:
    cs += u[s_names[-1]][vehicle] <= T-s[s_names[-1]][vehicle]

#finding distance travelled by each truck and ensuring it it possible
dist = 0
for i in range(len(s_names)):
    dist += stations[i]
    for vehicle in vehicles:
        cs += charging_rate_station*(pulp.lpSum(s[s_names[j]][vehicle] for j in range(i+1)))+ex_charging_rate_bess*(pulp.lpSum(b[s_names[j]][vehicle] for j in range(i+1))) + soc_vehicle[vehicle] >= dist + stations[i+1]
        cs += charging_rate_station*(pulp.lpSum(s[s_names[j]][vehicle] for j in range(i+1)))+ex_charging_rate_bess*(pulp.lpSum(b[s_names[j]][vehicle] for j in range(i+1))) + soc_vehicle[vehicle] -dist <= 100

for grouped_station in grouped_stations:
    
    for i in range(len(vehicles)):
        #calculating how much energy has been used and charged and the end of a vehicle.
        cs += bac[grouped_station][vehicles[i]] ==  pulp.lpSum([[(u[station][vehicles[i]]-u[station][vehicles[0]] - 2*pulp.lpSum([b[station][vehicles[j]]] for j in range(i-1,-1,-1)))*ex_charging_rate_bess] for station in grouped_stations[grouped_station]])
    for vehicle in vehicles: 
        #checking if the amount we charge is greater than 100 or not
        cs += M*charge[grouped_station][vehicle]>=bess_init_name[grouped_station]+bac[grouped_station][vehicle]-bess_capacity
        cs += M*(1-charge[grouped_station][vehicle]) >= -(bess_init_name[grouped_station]+bac[grouped_station][vehicle]-bess_capacity)
        #implementing that the stations cannot have less than zero bess charge left.
        cs += pulp.lpSum([b[station][vehicle]*ex_charging_rate_bess for station in grouped_stations[grouped_station]])<=bc[grouped_station][vehicle]
        # implementing max of bess capaicty
        cs += bc[grouped_station][vehicle]>= bess_capacity - M*(1-charge[grouped_station][vehicle])
        cs += bc[grouped_station][vehicle]>= bess_init_name[grouped_station] + bac[grouped_station][vehicle] - M*(charge[grouped_station][vehicle])
        cs += bc[grouped_station][vehicle]<= bess_capacity + M*(1-charge[grouped_station][vehicle])
        cs += bc[grouped_station][vehicle]<= bess_init_name[grouped_station] + bac[grouped_station][vehicle] + M*(charge[grouped_station][vehicle])  



#constraints
for station in s_names:
    for vehicle in vehicles:
        
        #departure = service + charge start
        cs += d[station][vehicle] == s[station][vehicle]+u[station][vehicle]

        # charge start > arival time
        cs += u[station][vehicle]>=a[station][vehicle]

        #to see if a vehicle wants to cross the station or charge there
        cs += M*sb[station][vehicle] >=s[station][vehicle]
        cs += sb[station][vehicle] <= s[station][vehicle]

        # bess time should be less than total charging time at 
        cs += b[station][vehicle]<=s[station][vehicle]   

    for vehicle1 in vehicles:
        for vehicle2 in vehicles:
            if vehicle1!=vehicle2:

                #making sure o[vehicle1+vehicle2] = 1 when vehicle1 in front of vehicle2
                cs += M*o[station][vehicle1+vehicle2] >= a[station][vehicle2]-a[station][vehicle1]
                cs += M*(1-o[station][vehicle1+vehicle2])>= a[station][vehicle1]-a[station][vehicle2]

                #implementing wait time
                cs += u[station][vehicle2]-u[station][vehicle1]-s[station][vehicle1]>=(o[station][vehicle1+vehicle2]-1)*T-M*(1-sb[station][vehicle2])                

                #making sure only one vehicle is front of another
                cs += o[station][vehicle1+vehicle2] + o[station][vehicle2+vehicle1] >= 1


{'station1': ['station1', 'station2'], 'station3': ['station3', 'station4']}
['station1', 'station2', 'station3', 'station4']
{'station1': 250, 'station3': 250}
['V0', 'V1', 'V2']
station1
station3
{'station1': {'V0': BESS_charge_for_vehicle_station1_V0, 'V1': BESS_charge_for_vehicle_station1_V1, 'V2': BESS_charge_for_vehicle_station1_V2}, 'station3': {'V0': BESS_charge_for_vehicle_station3_V0, 'V1': BESS_charge_for_vehicle_station3_V1, 'V2': BESS_charge_for_vehicle_station3_V2}} {'station1': {'V0': BESS_charge_station1_V0, 'V1': BESS_charge_station1_V1, 'V2': BESS_charge_station1_V2}, 'station3': {'V0': BESS_charge_station3_V0, 'V1': BESS_charge_station3_V1, 'V2': BESS_charge_station3_V2}} {'station1': {'V0': reached_max_charge_station1_V0, 'V1': reached_max_charge_station1_V1, 'V2': reached_max_charge_station1_V2}, 'station3': {'V0': reached_max_charge_station3_V0, 'V1': reached_max_charge_station3_V1, 'V2': reached_max_charge_station3_V2}}


Need to change the sigma variable to incorporate charging points.
need to change "u" in terms of charing points, "s" as in terms of charging points
"d" in terms of charging points,

In [98]:
cs

station:
MINIMIZE
None
SUBJECT TO
_C1: Arrival_Time_station1_V0 = 2

_C2: Arrival_Time_station1_V1 = 12

_C3: Arrival_Time_station1_V2 = 17

_C4: Arrival_Time_station2_V0 - final_charge_time_station1_V0 = 0

_C5: Arrival_Time_station2_V1 - final_charge_time_station1_V1 = 0

_C6: Arrival_Time_station2_V2 - final_charge_time_station1_V2 = 0

_C7: Arrival_Time_station3_V0 - final_charge_time_station2_V0 = 4

_C8: Arrival_Time_station3_V1 - final_charge_time_station2_V1 = 4

_C9: Arrival_Time_station3_V2 - final_charge_time_station2_V2 = 4

_C10: Arrival_Time_station4_V0 - final_charge_time_station3_V0 = 0

_C11: Arrival_Time_station4_V1 - final_charge_time_station3_V1 = 0

_C12: Arrival_Time_station4_V2 - final_charge_time_station3_V2 = 0

_C13: Service_Time_station4_V0 + initial_charge_time_station4_V0 <= 1000

_C14: Service_Time_station4_V1 + initial_charge_time_station4_V1 <= 1000

_C15: Service_Time_station4_V2 + initial_charge_time_station4_V2 <= 1000

_C16: 10 BESS_service_Time_stat

In [99]:
cs += pulp.lpSum([d[station][vehicle]-a[station][vehicle]] for station in s_names for vehicle in vehicles)
cs.solve()

pulp.LpStatus[cs.status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/sushi/Desktop/internship/Routing/.venv/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/5f447b483e7f475b807a44351f34a49b-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/5f447b483e7f475b807a44351f34a49b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 224 COLUMNS
At line 1148 RHS
At line 1368 BOUNDS
At line 1483 ENDATA
Problem MODEL has 219 rows, 114 columns and 667 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 12 - 0.00 seconds
Cgl0003I 0 fixed, 12 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 4 strengthened rows, 0 substitutions
Cgl0004I processed model has 143 rows, 79 columns (79 integer (40 of which binary)) and 481 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0045I 6 integer vari

'Optimal'

In [100]:
for grouped_station in grouped_stations:
    for station in grouped_stations[grouped_station]:
        for vehicle in vehicles:
            print(f"{vehicle} waits at {station} for {u[station][vehicle].varValue-a[station][vehicle].varValue}")
            print(f"{vehicle} charges at {station} for {b[station][vehicle].varValue} using the bess and station, and for {s[station][vehicle].varValue-b[station][vehicle].varValue} only using station")
            print(f"{vehicle} reaches {station} at {a[station][vehicle].varValue}")
            print(f"bess energy level for {vehicle} is {bc[grouped_station][vehicle].varValue} at {station}")
            print(f"{vehicle} charges {station} at {u[station][vehicle].varValue}")

            print()


V0 waits at station1 for 0.0
V0 charges at station1 for 0.0 using the bess and station, and for 0.0 only using station
V0 reaches station1 at 2.0
bess energy level for V0 is 0.0 at station1
V0 charges station1 at 2.0

V1 waits at station1 for 0.0
V1 charges at station1 for 0.0 using the bess and station, and for 0.0 only using station
V1 reaches station1 at 12.0
bess energy level for V1 is 0.0 at station1
V1 charges station1 at 12.0

V2 waits at station1 for 0.0
V2 charges at station1 for 0.0 using the bess and station, and for 0.0 only using station
V2 reaches station1 at 17.0
bess energy level for V2 is 0.0 at station1
V2 charges station1 at 17.0

V0 waits at station2 for 0.0
V0 charges at station2 for 0.0 using the bess and station, and for 0.0 only using station
V0 reaches station2 at 2.0
bess energy level for V0 is 0.0 at station2
V0 charges station2 at 2.0

V1 waits at station2 for 0.0
V1 charges at station2 for 0.0 using the bess and station, and for 0.0 only using station
V1 re

 best solution, best possible 54 (635.43 seconds)
Cbc0010I After 325000 nodes, 3891 on tree, 55 best solution, best possible 54 (637.14 seconds)
Cbc0010I After 326000 nodes, 3715 on tree, 55 best solution, best possible 54 (638.69 seconds)
Cbc0010I After 327000 nodes, 3544 on tree, 55 best solution, best possible 54 (640.51 seconds)
Cbc0010I After 328000 nodes, 3361 on tree, 55 best solution, best possible 54 (642.23 seconds)
Cbc0010I After 329000 nodes, 3213 on tree, 55 best solution, best possible 54 (643.93 seconds)
Cbc0010I After 330000 nodes, 3034 on tree, 55 best solution, best possible 54 (645.71 seconds)
Cbc0010I After 331000 nodes, 2867 on tree, 55 best solution, best possible 54 (647.85 seconds)
Cbc0010I After 332000 nodes, 2678 on tree, 55 best solution, best possible 54 (649.60 seconds)
Cbc0010I After 333000 nodes, 2511 on tree, 55 best solution, best possible 54 (651.27 seconds)
Cbc0010I After 334000 nodes, 2357 on tree, 55 best solution, best possible 54 (652.96 seconds)


In [95]:


#objective function
print("wait and charging time is ",pulp.value(cs.objective))

state = {}
for vehicle in vehicles:
    state[vehicle] = []
max_len = 0
for i in range(len(s_names)):
    for vehicle in vehicles:
        for l in range(int(stations[i]/speed)):
            state[vehicle].append("driving")
        for j in range(int(u[s_names[i]][vehicle].varValue-a[s_names[i]][vehicle].varValue)):
            state[vehicle].append("waiting")
        for k in range(int(s[s_names[i]][vehicle].varValue)):
            state[vehicle].append("charging")
for vehicle in vehicles:
    for l in range(int(stations[-1]/speed)):
        state[vehicle].append("driving")
    if len(state[vehicle])>max_len:
        max_len = len(state[vehicle])
for vehicle in vehicles:
    if len(state[vehicle]) < max_len:
        for l in range(max_len-len(state[vehicle])):
            state[vehicle].append("reached")


# state is first graph variable     max_ len is max time       
            
soc_trucks = {}
max_len_soc = 0
for vehicle in vehicles:
    soc_trucks[vehicle] = [soc_vehicle[vehicle]]
for i in range(len(s_names)):
    for vehicle in vehicles:
        for j in range(int(stations[i]/speed)):
            soc_trucks[vehicle].append(soc_trucks[vehicle][-1]-speed)
        for j in range(int(u[s_names[i]][vehicle].varValue-a[s_names[i]][vehicle].varValue)):
            soc_trucks[vehicle].append(soc_trucks[vehicle][-1])
        for k in range(int(b[s_names[i]][vehicle].varValue)):
            soc_trucks[vehicle].append(soc_trucks[vehicle][-1]+ex_charging_rate_bess+charging_rate_station)
        for j in range(int(s[s_names[i]][vehicle].varValue-b[s_names[i]][vehicle].varValue)):
            soc_trucks[vehicle].append(soc_trucks[vehicle][-1]+charging_rate_station)
for vehicle in vehicles:
    for j in range(int(stations[-1]/speed)):
        soc_trucks[vehicle].append(soc_trucks[vehicle][-1]-speed)
    if len(soc_trucks[vehicle])>max_len_soc:
        max_len_soc = len(soc_trucks[vehicle])
for vehicle in vehicles:
    if len(soc_trucks[vehicle]) < max_len_soc:
        for j in range(max_len_soc-len(soc_trucks[vehicle])):
            soc_trucks[vehicle].append(soc_trucks[vehicle][-1])
soc_bess = []
bess_state = []
for i in range(len(s_names)):
    soc_bess.append([bess_init[i]])
    bess_state.append([])
for i  in range(len(s_names)):
    for j  in range(len(vehicles)):
        if j == 0:
            for k in range(int(u[s_names[i]][vehicles[j]].varValue)):
                bess_state[i].append("idle")
                soc_bess[i].append(soc_bess[i][-1])
            for k in range(int(b[s_names[i]][vehicles[j]].varValue)):
                bess_state[i].append("discharging")
                soc_bess[i].append(soc_bess[i][-1]-ex_charging_rate_bess)
            for k in range(int(s[s_names[i]][vehicles[j]].varValue-b[s_names[i]][vehicles[j]].varValue)):
                if soc_bess[i][-1] >= bess_capacity:
                    bess_state[i].append("idle")
                    soc_bess[i].append(soc_bess[i][-1])
                else:
                    bess_state[i].append("charging")
                    soc_bess[i].append(min(bess_capacity,soc_bess[i][-1]+ex_charging_rate_bess))
        else:
            for k in range(int(u[s_names[i]][vehicles[j]].varValue-u[s_names[i]][vehicles[j-1]].varValue-b[s_names[i]][vehicles[j-1]].varValue)):
                if soc_bess[i][-1] >= bess_capacity:
                    bess_state[i].append("idle")
                    soc_bess[i].append(soc_bess[i][-1])
                else:
                    bess_state[i].append("charging")
                    soc_bess[i].append(min(bess_capacity,soc_bess[i][-1]+ex_charging_rate_bess))
            for k in range(int(b[s_names[i]][vehicles[j]].varValue)):
                bess_state[i].append("discharging")
                soc_bess[i].append(soc_bess[i][-1]-ex_charging_rate_bess)
            for k in range(int(s[s_names[i]][vehicles[j]].varValue-b[s_names[i]][vehicles[j]].varValue)):
                if soc_bess[i][-1] >= bess_capacity:
                    bess_state[i].append("idle")
                    soc_bess[i].append(soc_bess[i][-1])
                else:
                    bess_state[i].append("charging")
                    soc_bess[i].append(min(bess_capacity,soc_bess[i][-1]+ex_charging_rate_bess))
                    
for vehicle in vehicles:
    if len(state[vehicle]) < max_len:
        for i in range(max_len-len(state[vehicle])):
            state[vehicle].append("reached")
max_len_bess = 0
for i in range(len(s_names)):
    max_len_bess = max(max_len_bess,len(soc_bess[i]))
for i in range(len(soc_bess)):
    for j in range(max_len_bess-len(soc_bess[i])):
        bess_state[i].append("idle")
        soc_bess[i].append(soc_bess[i][-1])
for i in range(len(s_names)):
    soc_bess[i].pop(0)

wait_time = {}
for vehicle in  vehicles:
    wait_time[vehicle] = []
for vehicle in vehicles:
    for station in s_names:
        wait_time[vehicle].append((u[station][vehicle].varValue-a[station][vehicle].varValue)/time_discre)


import seaborn as sns
import matplotlib.pyplot as plt

plot = [[] for i in range(len(vehicles))]
for i in range(len(vehicles)):
    plot[i]=wait_time[vehicles[i]]

import numpy as np  
import matplotlib.pyplot as plt  
X =s_names[:]
plt.figure(figsize=(5,5))  
X_axis = np.arange(len(X)) 
for i in range(len(vehicles)):
    plt.bar(X_axis - 0.1*(i-(len(vehicles)//2)), plot[i], 0.1, label = vehicles[i])
plt.xticks(X_axis, X) 
plt.xlabel("station") 
plt.ylabel("Wait time") 
plt.title("Truck wait time") 
plt.legend() 
plt.show() 


plt.figure(figsize=(5,5))
plt.style.use('ggplot')
# create data 
x = list(range(len(state[vehicles[0]])))
for i in range(len(x)):
    x[i] = x[i]/time_discre 
# plot lines 
for vehicle in vehicles:
    plt.plot(x, state[vehicle], label = vehicle) 
plt.xlabel("Time") 
plt.ylabel("Truck state") 
plt.title("Truck schedule") 
plt.legend() 
plt.show() 

plt.figure(figsize=(5,5))

x = list(range(max_len_soc))
for i in range(len(x)):
    x[i] = x[i]/time_discre

# plot lines 
for vehicle in vehicles:
    plt.plot(x, soc_trucks[vehicle], label = vehicle)
plt.xlabel("Time") 
plt.ylabel("Truck State of Charge") 
plt.title("SoC of Trucks")  
plt.legend() 
plt.show()

plt.figure(figsize=(5,5))
x = list(range(len(bess_state[0]))) 
for i in range(len(x)):
    x[i] = x[i]/time_discre
# plot lines 
for i in range(len(s_names)):
    plt.plot(x, bess_state[i], label = f"Station{i+1}") 
plt.xlabel("Time") 
plt.ylabel("bess state") 
plt.title("bess schedule") 
plt.legend() 
plt.show() 

plt.figure(figsize=(5,5))

x = list(range(len(soc_bess[0])))
for i in range(len(x)):
    x[i] = x[i]/time_discre
# plot lines 
for i in range(len(s_names)):
    plt.plot(x, soc_bess[i], label = f"Station{i+1}") 
plt.xlabel("Time") 
plt.ylabel("BESS State of Charge") 
plt.title("SoC of BESS")  
plt.legend() 
plt.show()


wait and charging time is  12.0


IndexError: list index out of range

In [96]:
pulp.LpStatus[cs.status]

'Optimal'

 best possible 53.268566 (571.39 seconds)
Cbc0010I After 286000 nodes, 9004 on tree, 55 best solution, best possible 53.268566 (577.12 seconds)
Cbc0010I After 287000 nodes, 9072 on tree, 55 best solution, best possible 53.333323 (578.48 seconds)
Cbc0010I After 288000 nodes, 9054 on tree, 55 best solution, best possible 53.333333 (579.89 seconds)
Cbc0010I After 289000 nodes, 9018 on tree, 55 best solution, best possible 53.357143 (581.22 seconds)
Cbc0010I After 290000 nodes, 8865 on tree, 55 best solution, best possible 53.357143 (582.49 seconds)
Cbc0010I After 291000 nodes, 8896 on tree, 55 best solution, best possible 53.4004 (583.69 seconds)
Cbc0010I After 292000 nodes, 8870 on tree, 55 best solution, best possible 53.446225 (585.12 seconds)
Cbc0010I After 293000 nodes, 8878 on tree, 55 best solution, best possible 53.499999 (586.52 seconds)
Cbc0010I After 294000 nodes, 8685 on tree, 55 best solution, best possible 53.499999 (588.17 seconds)
Cbc0010I After 295000 nodes, 8657 on tree,